# Notebook 5: Data Merging & Final Dataset Construction
## Creating the Event-Level Analysis Dataset

---

**Research Project:** Retail Sentiment, Earnings Quality, and Stock Returns

**Purpose:** Merge all data layers to create the final analysis dataset.

**Input Files:**
- `wsb_firm_day_panel.parquet` - Social media sentiment
- `stock_returns_panel.parquet` - Stock returns & characteristics
- `earnings_quality_panel.parquet` - Earnings quality measures
- Earnings announcement dates

**Output:** Event-level panel linking:
- Pre-EA social media sentiment and attention
- EA abnormal returns (CAR)
- Post-EA drift returns
- Firm-level earnings quality
- Control variables

---

## 1. Environment Setup

In [ ]:
# =============================================================================
# INSTALL REQUIRED PACKAGES
# =============================================================================

!pip install pandas==2.0.3
!pip install numpy==1.24.3
!pip install scipy==1.11.3
!pip install pyarrow==14.0.1
!pip install tqdm==4.66.1
!pip install yfinance==0.2.31

print("All packages installed successfully.")

In [ ]:
# =============================================================================
# IMPORT LIBRARIES
# =============================================================================

import os
import json
import warnings
from datetime import datetime, timedelta
from typing import List, Dict, Tuple, Optional

import pandas as pd
import numpy as np
from scipy import stats
from tqdm.notebook import tqdm
import yfinance as yf

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.4f}'.format)

print(f"Environment setup complete. Timestamp: {datetime.now()}")

In [ ]:
# =============================================================================
# CONFIGURATION
# =============================================================================

class MergeConfig:
    """Configuration for data merging."""
    
    # Data paths
    BASE_PATH = "/content/drive/MyDrive/Research/RetailSentiment/"
    PROCESSED_DATA_PATH = BASE_PATH + "data/processed/"
    FINAL_DATA_PATH = BASE_PATH + "data/final/"
    
    # Event windows (in trading days)
    PRE_EA_WINDOW_START = -10  # Start of pre-EA sentiment window
    PRE_EA_WINDOW_END = -2     # End of pre-EA sentiment window
    
    EA_WINDOW_START = -1       # EA return window start
    EA_WINDOW_END = 1          # EA return window end
    
    POST_EA_WINDOW_START = 2   # Post-EA drift start
    POST_EA_WINDOW_END = 20    # Post-EA drift end
    
    # Sample filters
    MIN_POSTS_PRE_EA = 1       # Minimum posts in pre-EA window
    MIN_MARKET_CAP = 1e8      # Minimum market cap ($100M)
    MIN_PRICE = 5.0           # Minimum stock price
    
    @classmethod
    def print_config(cls):
        print("="*60)
        print("DATA MERGE CONFIGURATION")
        print("="*60)
        print(f"Pre-EA window: [{cls.PRE_EA_WINDOW_START}, {cls.PRE_EA_WINDOW_END}]")
        print(f"EA window: [{cls.EA_WINDOW_START}, {cls.EA_WINDOW_END}]")
        print(f"Post-EA window: [{cls.POST_EA_WINDOW_START}, {cls.POST_EA_WINDOW_END}]")
        print(f"Min posts pre-EA: {cls.MIN_POSTS_PRE_EA}")
        print(f"Min market cap: ${cls.MIN_MARKET_CAP/1e6:.0f}M")
        print("="*60)

config = MergeConfig()
config.print_config()

In [ ]:
# =============================================================================
# MOUNT GOOGLE DRIVE
# =============================================================================

from google.colab import drive
drive.mount('/content/drive')

os.makedirs(config.FINAL_DATA_PATH, exist_ok=True)
print("Data directories ready.")

## 2. Load All Data Sources

In [ ]:
# =============================================================================
# DATA LOADER
# =============================================================================

class DataLoader:
    """Loads and validates all input data sources."""
    
    def __init__(self, data_path: str):
        self.data_path = data_path
        self.data = {}
        
    def load_social_media(self) -> pd.DataFrame:
        """Load social media sentiment panel."""
        filepath = os.path.join(self.data_path, 'wsb_firm_day_panel.parquet')
        
        if os.path.exists(filepath):
            df = pd.read_parquet(filepath)
            df['date'] = pd.to_datetime(df['date'])
            print(f"Loaded social media panel: {len(df):,} obs, {df['ticker'].nunique()} firms")
            return df
        else:
            print(f"Warning: {filepath} not found. Creating synthetic data.")
            return self._create_synthetic_sentiment()
    
    def load_stock_returns(self) -> pd.DataFrame:
        """Load stock returns panel."""
        filepath = os.path.join(self.data_path, 'stock_returns_panel.parquet')
        
        if os.path.exists(filepath):
            df = pd.read_parquet(filepath)
            df['date'] = pd.to_datetime(df['date'])
            print(f"Loaded stock returns: {len(df):,} obs, {df['ticker'].nunique()} firms")
            return df
        else:
            print(f"Warning: {filepath} not found. Creating synthetic data.")
            return self._create_synthetic_returns()
    
    def load_earnings_quality(self) -> pd.DataFrame:
        """Load earnings quality panel."""
        filepath = os.path.join(self.data_path, 'earnings_quality_panel.parquet')
        
        if os.path.exists(filepath):
            df = pd.read_parquet(filepath)
            df['period_end'] = pd.to_datetime(df['period_end'])
            print(f"Loaded earnings quality: {len(df):,} obs, {df['ticker'].nunique()} firms")
            return df
        else:
            print(f"Warning: {filepath} not found. Creating synthetic data.")
            return self._create_synthetic_eq()
    
    def load_firm_characteristics(self) -> pd.DataFrame:
        """Load firm characteristics."""
        filepath = os.path.join(self.data_path, 'firm_characteristics.parquet')
        
        if os.path.exists(filepath):
            df = pd.read_parquet(filepath)
            print(f"Loaded firm characteristics: {len(df)} firms")
            return df
        else:
            return pd.DataFrame()
    
    def _create_synthetic_sentiment(self) -> pd.DataFrame:
        """Create synthetic sentiment data for demonstration."""
        np.random.seed(42)
        tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'META', 'TSLA', 'NVDA', 
                   'GME', 'AMC', 'BB', 'PLTR', 'NIO', 'SPY', 'QQQ', 'AMD']
        dates = pd.date_range('2020-01-01', '2023-12-31', freq='D')
        
        data = []
        for ticker in tickers:
            for date in dates:
                if np.random.random() > 0.3:  # 70% coverage
                    data.append({
                        'ticker': ticker,
                        'date': date,
                        'PostCount': np.random.poisson(10),
                        'UniqueUsers': np.random.poisson(8),
                        'SentimentMean': np.random.normal(0.1, 0.3),
                        'SentimentPosShare': np.random.uniform(0.2, 0.6),
                        'SentimentNegShare': np.random.uniform(0.1, 0.4),
                        'SentimentStd': np.random.uniform(0.1, 0.5),
                        'KarmaSum': np.random.poisson(100),
                        'Attention': np.random.uniform(1, 4)
                    })
        
        return pd.DataFrame(data)
    
    def _create_synthetic_returns(self) -> pd.DataFrame:
        """Create synthetic returns data for demonstration."""
        np.random.seed(42)
        tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'META', 'TSLA', 'NVDA', 
                   'GME', 'AMC', 'BB', 'PLTR', 'NIO', 'SPY', 'QQQ', 'AMD']
        dates = pd.bdate_range('2020-01-01', '2023-12-31')
        
        data = []
        for ticker in tickers:
            price = 100
            for date in dates:
                ret = np.random.normal(0.0005, 0.02)
                price = price * (1 + ret)
                data.append({
                    'ticker': ticker,
                    'date': date,
                    'close': price,
                    'volume': np.random.poisson(1e7),
                    'ret': ret,
                    'market_return': np.random.normal(0.0004, 0.01),
                    'ret_mktadj': ret - np.random.normal(0.0004, 0.01)
                })
        
        return pd.DataFrame(data)
    
    def _create_synthetic_eq(self) -> pd.DataFrame:
        """Create synthetic earnings quality data."""
        np.random.seed(42)
        tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'META', 'TSLA', 'NVDA', 
                   'GME', 'AMC', 'BB', 'PLTR', 'NIO', 'SPY', 'QQQ', 'AMD']
        quarters = pd.date_range('2020-01-01', '2023-12-31', freq='Q')
        
        data = []
        for ticker in tickers:
            base_eq = np.random.normal(0, 1)
            for q in quarters:
                data.append({
                    'ticker': ticker,
                    'period_end': q,
                    'earnings_quality_dd_std': base_eq + np.random.normal(0, 0.2),
                    'earnings_quality_mcn_std': base_eq + np.random.normal(0, 0.25),
                    'earnings_quality_composite': base_eq + np.random.normal(0, 0.15)
                })
        
        return pd.DataFrame(data)

# Initialize loader
loader = DataLoader(config.PROCESSED_DATA_PATH)

# Load all data
print("Loading all data sources...\n")
social_media = loader.load_social_media()
stock_returns = loader.load_stock_returns()
earnings_quality = loader.load_earnings_quality()
firm_chars = loader.load_firm_characteristics()

## 3. Create Earnings Announcement Events

In [ ]:
# =============================================================================
# EARNINGS ANNOUNCEMENT EVENT CREATOR
# =============================================================================

class EarningsEventCreator:
    """Creates earnings announcement event observations."""
    
    def __init__(self, config: MergeConfig):
        self.config = config
        
    def get_earnings_dates(self, tickers: List[str]) -> pd.DataFrame:
        """Fetch earnings announcement dates from Yahoo Finance.
        
        Args:
            tickers: List of ticker symbols
            
        Returns:
            DataFrame with earnings events
        """
        print(f"Fetching earnings dates for {len(tickers)} tickers...")
        
        events = []
        
        for ticker in tqdm(tickers, desc="Fetching EA dates"):
            try:
                stock = yf.Ticker(ticker)
                earnings = stock.earnings_dates
                
                if earnings is not None and len(earnings) > 0:
                    earnings_df = earnings.reset_index()
                    earnings_df.columns = ['ea_datetime'] + list(earnings_df.columns[1:])
                    
                    for _, row in earnings_df.iterrows():
                        events.append({
                            'ticker': ticker,
                            'ea_date': row['ea_datetime'].date() if hasattr(row['ea_datetime'], 'date') else row['ea_datetime'],
                            'eps_estimate': row.get('EPS Estimate', np.nan),
                            'reported_eps': row.get('Reported EPS', np.nan),
                            'surprise_pct': row.get('Surprise(%)', np.nan)
                        })
            except:
                continue
        
        df = pd.DataFrame(events)
        if len(df) > 0:
            df['ea_date'] = pd.to_datetime(df['ea_date'])
            print(f"Collected {len(df)} earnings events")
        
        return df
    
    def create_synthetic_events(self, tickers: List[str],
                                start_date: str = '2020-01-01',
                                end_date: str = '2023-12-31') -> pd.DataFrame:
        """Create synthetic earnings events for demonstration.
        
        Assumes quarterly earnings with some randomization.
        """
        print("Creating synthetic earnings events...")
        np.random.seed(42)
        
        events = []
        quarters = pd.date_range(start_date, end_date, freq='Q')
        
        for ticker in tickers:
            for q_end in quarters:
                # EA typically 2-6 weeks after quarter end
                ea_date = q_end + pd.Timedelta(days=np.random.randint(14, 42))
                
                # Random surprise
                surprise = np.random.normal(0.02, 0.15)  # 2% mean, 15% std
                
                events.append({
                    'ticker': ticker,
                    'ea_date': ea_date,
                    'quarter_end': q_end,
                    'eps_estimate': np.random.uniform(0.5, 3.0),
                    'reported_eps': None,  # Will be calculated
                    'surprise_pct': surprise * 100
                })
        
        df = pd.DataFrame(events)
        df['reported_eps'] = df['eps_estimate'] * (1 + df['surprise_pct']/100)
        df['ea_date'] = pd.to_datetime(df['ea_date'])
        df['quarter_end'] = pd.to_datetime(df['quarter_end'])
        
        print(f"Created {len(df)} synthetic earnings events")
        return df

# Initialize event creator
event_creator = EarningsEventCreator(config)

# Get unique tickers from data
all_tickers = list(set(social_media['ticker'].unique()) & 
                   set(stock_returns['ticker'].unique()))
print(f"Common tickers: {len(all_tickers)}")

# Get earnings events
# Try real data first, fall back to synthetic
try:
    ea_events = event_creator.get_earnings_dates(all_tickers[:20])  # Subset for demo
    if len(ea_events) < 50:
        raise ValueError("Insufficient real data")
except:
    ea_events = event_creator.create_synthetic_events(all_tickers)

print(f"\nEarnings events: {len(ea_events)}")
print(ea_events.head())

## 4. Calculate Event-Window Variables

### 4.1 Pre-EA Social Media Aggregation

In [ ]:
# =============================================================================
# PRE-EA SENTIMENT AGGREGATOR
# =============================================================================

class PreEASentimentAggregator:
    """Aggregates social media variables over pre-EA window."""
    
    def __init__(self, config: MergeConfig):
        self.config = config
        
    def aggregate_sentiment(
        self,
        events: pd.DataFrame,
        sentiment_panel: pd.DataFrame,
        returns_panel: pd.DataFrame
    ) -> pd.DataFrame:
        """Aggregate sentiment over pre-EA window for each event.
        
        Args:
            events: Earnings announcement events
            sentiment_panel: Daily firm sentiment data
            returns_panel: Stock returns (for trading day calculation)
            
        Returns:
            Events with pre-EA sentiment variables
        """
        print("Aggregating pre-EA sentiment...")
        
        results = []
        
        # Create trading day calendar
        trading_days = returns_panel['date'].sort_values().unique()
        trading_day_idx = {d: i for i, d in enumerate(trading_days)}
        
        for _, event in tqdm(events.iterrows(), total=len(events), desc="Processing events"):
            ticker = event['ticker']
            ea_date = event['ea_date']
            
            # Find EA date in trading calendar
            ea_date_np = np.datetime64(ea_date)
            idx = np.searchsorted(trading_days, ea_date_np)
            
            if idx == 0 or idx >= len(trading_days):
                continue
            
            # Get pre-EA window dates
            start_idx = max(0, idx + self.config.PRE_EA_WINDOW_START)
            end_idx = idx + self.config.PRE_EA_WINDOW_END
            
            if end_idx < 0 or start_idx >= len(trading_days):
                continue
            
            window_start = pd.Timestamp(trading_days[start_idx])
            window_end = pd.Timestamp(trading_days[end_idx])
            
            # Filter sentiment data
            mask = (
                (sentiment_panel['ticker'] == ticker) &
                (sentiment_panel['date'] >= window_start) &
                (sentiment_panel['date'] <= window_end)
            )
            window_data = sentiment_panel[mask]
            
            # Aggregate
            result = {
                'ticker': ticker,
                'ea_date': ea_date,
                'window_start': window_start,
                'window_end': window_end,
                'n_days_with_posts': len(window_data),
            }
            
            if len(window_data) > 0:
                # Attention measures
                result['pre_ea_posts'] = window_data['PostCount'].sum()
                result['pre_ea_users'] = window_data['UniqueUsers'].sum()
                result['pre_ea_attention'] = np.log1p(result['pre_ea_posts'])
                result['pre_ea_attention_avg'] = window_data['Attention'].mean() if 'Attention' in window_data.columns else np.nan
                
                # Sentiment measures
                result['pre_ea_sentiment_mean'] = window_data['SentimentMean'].mean()
                result['pre_ea_sentiment_median'] = window_data['SentimentMean'].median()
                result['pre_ea_pos_share'] = window_data['SentimentPosShare'].mean()
                result['pre_ea_neg_share'] = window_data['SentimentNegShare'].mean()
                result['pre_ea_sentiment_std'] = window_data['SentimentMean'].std()
                result['pre_ea_disagreement'] = window_data['SentimentStd'].mean() if 'SentimentStd' in window_data.columns else np.nan
                
                # Karma-weighted sentiment
                if 'KarmaSum' in window_data.columns:
                    total_karma = window_data['KarmaSum'].sum()
                    if total_karma > 0:
                        result['pre_ea_sentiment_karma_wtd'] = (
                            (window_data['SentimentMean'] * window_data['KarmaSum']).sum() / total_karma
                        )
                    else:
                        result['pre_ea_sentiment_karma_wtd'] = result['pre_ea_sentiment_mean']
            else:
                # No posts in window
                result['pre_ea_posts'] = 0
                result['pre_ea_users'] = 0
                result['pre_ea_attention'] = 0
                result['pre_ea_sentiment_mean'] = np.nan
            
            # Add original event data
            for col in ['quarter_end', 'eps_estimate', 'reported_eps', 'surprise_pct']:
                if col in event.index:
                    result[col] = event[col]
            
            results.append(result)
        
        df = pd.DataFrame(results)
        print(f"\nPre-EA sentiment aggregated for {len(df)} events")
        print(f"Events with posts: {(df['pre_ea_posts'] > 0).sum()}")
        
        return df

# Aggregate pre-EA sentiment
sentiment_agg = PreEASentimentAggregator(config)
events_with_sentiment = sentiment_agg.aggregate_sentiment(
    ea_events,
    social_media,
    stock_returns
)

### 4.2 Calculate Cumulative Abnormal Returns

In [ ]:
# =============================================================================
# CAR CALCULATOR FOR EVENTS
# =============================================================================

class EventCARCalculator:
    """Calculates CARs for earnings announcement events."""
    
    def __init__(self, config: MergeConfig):
        self.config = config
        
    def calculate_cars(
        self,
        events: pd.DataFrame,
        returns_panel: pd.DataFrame
    ) -> pd.DataFrame:
        """Calculate CARs for various windows around EA.
        
        Windows:
        - CAR[-1,+1]: EA announcement return
        - CAR[0,+2]: Immediate post-EA return
        - CAR[+2,+20]: Post-EA drift
        - CAR[-10,-2]: Pre-EA return (control)
        
        Args:
            events: Earnings events with sentiment
            returns_panel: Stock returns panel
            
        Returns:
            Events with CAR variables
        """
        print("Calculating CARs...")
        
        # Define windows
        windows = [
            ('CAR_m1_p1', -1, 1),     # EA window
            ('CAR_0_p2', 0, 2),        # Immediate post
            ('CAR_p2_p20', 2, 20),     # Drift
            ('CAR_m10_m2', -10, -2),   # Pre-EA
            ('CAR_m5_p5', -5, 5),      # Extended EA
        ]
        
        # Get trading days
        trading_days = returns_panel['date'].sort_values().unique()
        
        results = []
        
        for _, event in tqdm(events.iterrows(), total=len(events), desc="Calculating CARs"):
            ticker = event['ticker']
            ea_date = pd.Timestamp(event['ea_date'])
            
            # Get ticker returns
            ticker_returns = returns_panel[
                returns_panel['ticker'] == ticker
            ].set_index('date').sort_index()
            
            if len(ticker_returns) == 0:
                continue
            
            # Find EA date index
            ea_date_np = np.datetime64(ea_date)
            idx = np.searchsorted(trading_days, ea_date_np)
            
            result = event.to_dict()
            
            # Calculate each window
            for window_name, start_offset, end_offset in windows:
                start_idx = idx + start_offset
                end_idx = idx + end_offset + 1
                
                if start_idx < 0 or end_idx > len(trading_days):
                    result[window_name] = np.nan
                    result[f'{window_name}_raw'] = np.nan
                    continue
                
                window_dates = trading_days[start_idx:end_idx]
                
                # Get returns in window
                window_returns = ticker_returns[
                    ticker_returns.index.isin(window_dates)
                ]
                
                if len(window_returns) > 0:
                    # Market-adjusted CAR
                    if 'ret_mktadj' in window_returns.columns:
                        result[window_name] = window_returns['ret_mktadj'].sum()
                    else:
                        result[window_name] = window_returns['ret'].sum() - window_returns.get('market_return', 0).sum()
                    
                    # Raw return
                    result[f'{window_name}_raw'] = window_returns['ret'].sum()
                else:
                    result[window_name] = np.nan
                    result[f'{window_name}_raw'] = np.nan
            
            # Add volatility around EA
            vol_window_dates = trading_days[max(0, idx-20):min(len(trading_days), idx+20)]
            vol_returns = ticker_returns[ticker_returns.index.isin(vol_window_dates)]
            result['ea_volatility'] = vol_returns['ret'].std() * np.sqrt(252) if len(vol_returns) > 5 else np.nan
            
            results.append(result)
        
        df = pd.DataFrame(results)
        print(f"\nCARs calculated for {len(df)} events")
        
        # Print summary
        print("\nCAR Summary Statistics:")
        for window_name, _, _ in windows:
            if window_name in df.columns:
                print(f"  {window_name}: mean={df[window_name].mean():.4f}, std={df[window_name].std():.4f}")
        
        return df

# Calculate CARs
car_calculator = EventCARCalculator(config)
events_with_cars = car_calculator.calculate_cars(
    events_with_sentiment,
    stock_returns
)

## 5. Merge with Earnings Quality

In [ ]:
# =============================================================================
# MERGE EARNINGS QUALITY
# =============================================================================

def merge_earnings_quality(
    events: pd.DataFrame,
    eq_panel: pd.DataFrame
) -> pd.DataFrame:
    """Merge earnings quality measures with events.
    
    Match EQ from most recent quarter before EA date.
    
    Args:
        events: Event-level data
        eq_panel: Earnings quality panel
        
    Returns:
        Events with EQ variables
    """
    print("Merging earnings quality...")
    
    # Ensure date columns are datetime
    events = events.copy()
    events['ea_date'] = pd.to_datetime(events['ea_date'])
    eq_panel['period_end'] = pd.to_datetime(eq_panel['period_end'])
    
    # For each event, find most recent EQ
    merged_events = []
    
    for _, event in events.iterrows():
        ticker = event['ticker']
        ea_date = event['ea_date']
        
        # Get EQ data for ticker before EA date
        ticker_eq = eq_panel[
            (eq_panel['ticker'] == ticker) &
            (eq_panel['period_end'] < ea_date)
        ].sort_values('period_end', ascending=False)
        
        result = event.to_dict()
        
        if len(ticker_eq) > 0:
            # Take most recent
            latest_eq = ticker_eq.iloc[0]
            
            # Add EQ variables
            eq_cols = [c for c in latest_eq.index if 'earnings_quality' in c]
            for col in eq_cols:
                result[col] = latest_eq[col]
            
            result['eq_period_end'] = latest_eq['period_end']
            result['eq_lag_days'] = (ea_date - latest_eq['period_end']).days
        
        merged_events.append(result)
    
    df = pd.DataFrame(merged_events)
    
    # Create EQ quintiles for analysis
    if 'earnings_quality_composite' in df.columns:
        df['eq_quintile'] = pd.qcut(
            df['earnings_quality_composite'].rank(method='first'),
            5, labels=['Q1_Low', 'Q2', 'Q3', 'Q4', 'Q5_High']
        )
        df['eq_high'] = (df['eq_quintile'].isin(['Q4', 'Q5_High'])).astype(int)
        df['eq_low'] = (df['eq_quintile'].isin(['Q1_Low', 'Q2'])).astype(int)
    
    print(f"Merged EQ for {df['earnings_quality_composite'].notna().sum()} events")
    
    return df

# Merge earnings quality
events_with_eq = merge_earnings_quality(events_with_cars, earnings_quality)

## 6. Add Control Variables

In [ ]:
# =============================================================================
# ADD CONTROL VARIABLES
# =============================================================================

def add_control_variables(
    events: pd.DataFrame,
    returns_panel: pd.DataFrame,
    firm_chars: pd.DataFrame = None
) -> pd.DataFrame:
    """Add control variables to event data.
    
    Controls:
    - Size (log market cap)
    - Book-to-market
    - Prior returns (momentum)
    - Volatility
    - Liquidity
    - Sector
    - Announcement timing
    
    Args:
        events: Event data
        returns_panel: Stock returns
        firm_chars: Firm characteristics (optional)
        
    Returns:
        Events with control variables
    """
    print("Adding control variables...")
    
    df = events.copy()
    
    # Time-varying controls from returns panel
    controls = []
    
    for _, event in df.iterrows():
        ticker = event['ticker']
        ea_date = pd.Timestamp(event['ea_date'])
        
        # Get returns before EA
        ticker_returns = returns_panel[
            (returns_panel['ticker'] == ticker) &
            (returns_panel['date'] < ea_date)
        ].sort_values('date', ascending=False)
        
        result = {'ticker': ticker, 'ea_date': ea_date}
        
        if len(ticker_returns) > 0:
            latest = ticker_returns.iloc[0]
            
            # Price at EA-2
            result['price_pre_ea'] = latest['close']
            result['log_price'] = np.log(latest['close']) if latest['close'] > 0 else np.nan
            
            # Volume
            result['volume_pre_ea'] = latest['volume']
            result['log_volume'] = np.log1p(latest['volume'])
            
            # Prior returns (if available)
            if 'ret_1m' in latest.index:
                result['ret_1m'] = latest['ret_1m']
            if 'ret_3m' in latest.index:
                result['ret_3m'] = latest['ret_3m']
            if 'ret_6m' in latest.index:
                result['ret_6m'] = latest['ret_6m']
                
            # Volatility
            if len(ticker_returns) >= 20:
                result['volatility_pre_ea'] = ticker_returns.head(20)['ret'].std() * np.sqrt(252)
            else:
                result['volatility_pre_ea'] = ticker_returns['ret'].std() * np.sqrt(252)
            
            # Illiquidity
            if 'illiquidity_avg' in latest.index:
                result['illiquidity'] = latest['illiquidity_avg']
        
        controls.append(result)
    
    controls_df = pd.DataFrame(controls)
    
    # Merge controls
    df = df.merge(
        controls_df,
        on=['ticker', 'ea_date'],
        how='left'
    )
    
    # Merge firm characteristics if available
    if firm_chars is not None and len(firm_chars) > 0:
        firm_cols = ['ticker', 'sector', 'industry', 'market_cap', 'beta']
        firm_cols = [c for c in firm_cols if c in firm_chars.columns]
        
        df = df.merge(
            firm_chars[firm_cols],
            on='ticker',
            how='left'
        )
        
        # Log market cap
        if 'market_cap' in df.columns:
            df['log_mcap'] = np.log(df['market_cap'])
    
    # Time controls
    df['ea_year'] = df['ea_date'].dt.year
    df['ea_quarter'] = df['ea_date'].dt.quarter
    df['ea_month'] = df['ea_date'].dt.month
    df['ea_dayofweek'] = df['ea_date'].dt.dayofweek
    
    print(f"Added controls for {len(df)} events")
    
    return df

# Add control variables
final_events = add_control_variables(
    events_with_eq,
    stock_returns,
    firm_chars
)

## 7. Final Sample Filters and Validation

In [ ]:
# =============================================================================
# SAMPLE FILTERS AND VALIDATION
# =============================================================================

def apply_sample_filters(df: pd.DataFrame, config: MergeConfig) -> pd.DataFrame:
    """Apply sample filters for clean analysis dataset.
    
    Filters:
    1. Minimum WSB coverage (posts in pre-EA window)
    2. Minimum price
    3. Minimum market cap (if available)
    4. Non-missing key variables
    
    Args:
        df: Raw event dataset
        config: Configuration with filter parameters
        
    Returns:
        Filtered dataset
    """
    print("\nApplying sample filters...")
    print(f"Initial observations: {len(df):,}")
    
    filters_applied = []
    
    # Filter 1: Minimum WSB posts
    if 'pre_ea_posts' in df.columns:
        n_before = len(df)
        df = df[df['pre_ea_posts'] >= config.MIN_POSTS_PRE_EA]
        filters_applied.append(f"Min posts ({config.MIN_POSTS_PRE_EA}): -{n_before - len(df):,}")
    
    # Filter 2: Minimum price
    if 'price_pre_ea' in df.columns:
        n_before = len(df)
        df = df[df['price_pre_ea'] >= config.MIN_PRICE]
        filters_applied.append(f"Min price (${config.MIN_PRICE}): -{n_before - len(df):,}")
    
    # Filter 3: Non-missing CAR
    if 'CAR_m1_p1' in df.columns:
        n_before = len(df)
        df = df[df['CAR_m1_p1'].notna()]
        filters_applied.append(f"Missing CAR: -{n_before - len(df):,}")
    
    # Filter 4: Non-missing earnings quality
    if 'earnings_quality_composite' in df.columns:
        n_before = len(df)
        df = df[df['earnings_quality_composite'].notna()]
        filters_applied.append(f"Missing EQ: -{n_before - len(df):,}")
    
    print("\nFilters Applied:")
    for f in filters_applied:
        print(f"  {f}")
    
    print(f"\nFinal sample: {len(df):,} observations")
    print(f"Unique firms: {df['ticker'].nunique()}")
    
    return df.reset_index(drop=True)

# Apply filters
analysis_sample = apply_sample_filters(final_events, config)

In [ ]:
# =============================================================================
# DATA VALIDATION AND SUMMARY
# =============================================================================

def validate_and_summarize(df: pd.DataFrame):
    """Validate data quality and print summary statistics."""
    
    print("\n" + "="*70)
    print("FINAL DATASET VALIDATION AND SUMMARY")
    print("="*70)
    
    # Dimensions
    print(f"\n--- Sample Dimensions ---")
    print(f"Total observations: {len(df):,}")
    print(f"Unique firms: {df['ticker'].nunique()}")
    print(f"Date range: {df['ea_date'].min()} to {df['ea_date'].max()}")
    
    # Key variables
    print(f"\n--- Key Variables ---")
    
    # Social media variables
    sentiment_cols = [c for c in df.columns if 'pre_ea' in c and 'sentiment' in c]
    print("\nSocial Media Variables:")
    for col in sentiment_cols[:5]:
        print(f"  {col}: mean={df[col].mean():.4f}, std={df[col].std():.4f}, N={df[col].notna().sum()}")
    
    # CAR variables
    car_cols = [c for c in df.columns if c.startswith('CAR_') and '_raw' not in c]
    print("\nCAR Variables:")
    for col in car_cols:
        print(f"  {col}: mean={df[col].mean():.4f}, std={df[col].std():.4f}")
    
    # Earnings quality
    eq_cols = [c for c in df.columns if 'earnings_quality' in c]
    print("\nEarnings Quality Variables:")
    for col in eq_cols[:3]:
        print(f"  {col}: mean={df[col].mean():.4f}, std={df[col].std():.4f}")
    
    # Correlations
    print("\n--- Key Correlations ---")
    key_vars = ['pre_ea_sentiment_mean', 'CAR_m1_p1', 'earnings_quality_composite', 'pre_ea_posts']
    key_vars = [v for v in key_vars if v in df.columns]
    if len(key_vars) > 1:
        print(df[key_vars].corr().to_string())
    
    # Missing values
    print("\n--- Missing Values (>5%) ---")
    missing = df.isnull().sum()
    missing_pct = 100 * missing / len(df)
    significant_missing = missing_pct[missing_pct > 5].sort_values(ascending=False)
    if len(significant_missing) > 0:
        for col, pct in significant_missing.items():
            print(f"  {col}: {pct:.1f}%")
    else:
        print("  No variables with >5% missing")
    
    print("\n" + "="*70)

# Validate
validate_and_summarize(analysis_sample)

## 8. Create Interaction and Derived Variables

In [ ]:
# =============================================================================
# CREATE INTERACTION AND DERIVED VARIABLES
# =============================================================================

def create_analysis_variables(df: pd.DataFrame) -> pd.DataFrame:
    """Create interaction terms and derived variables for analysis.
    
    Key interactions:
    - Sentiment × EQ
    - Attention × EQ
    - Sentiment × Surprise
    
    Args:
        df: Event-level dataset
        
    Returns:
        Dataset with analysis variables
    """
    print("Creating analysis variables...")
    df = df.copy()
    
    # Standardize key variables
    def standardize(series):
        return (series - series.mean()) / series.std()
    
    # Standardized sentiment
    if 'pre_ea_sentiment_mean' in df.columns:
        df['sentiment_std'] = standardize(df['pre_ea_sentiment_mean'])
        df['sentiment_high'] = (df['sentiment_std'] > 0.5).astype(int)
        df['sentiment_low'] = (df['sentiment_std'] < -0.5).astype(int)
    
    # Standardized attention
    if 'pre_ea_attention' in df.columns:
        df['attention_std'] = standardize(df['pre_ea_attention'])
        df['attention_high'] = (df['attention_std'] > 0.5).astype(int)
    
    # Standardized EQ
    if 'earnings_quality_composite' in df.columns:
        df['eq_std'] = standardize(df['earnings_quality_composite'])
    
    # Key interaction: Sentiment × EQ
    if 'sentiment_std' in df.columns and 'eq_std' in df.columns:
        df['sentiment_x_eq'] = df['sentiment_std'] * df['eq_std']
        df['sentiment_x_eq_high'] = df['sentiment_std'] * df.get('eq_high', 0)
        df['sentiment_x_eq_low'] = df['sentiment_std'] * df.get('eq_low', 0)
    
    # Attention × EQ
    if 'attention_std' in df.columns and 'eq_std' in df.columns:
        df['attention_x_eq'] = df['attention_std'] * df['eq_std']
    
    # Surprise interactions
    if 'surprise_pct' in df.columns:
        df['surprise_std'] = standardize(df['surprise_pct'])
        df['surprise_positive'] = (df['surprise_pct'] > 0).astype(int)
        df['surprise_negative'] = (df['surprise_pct'] < 0).astype(int)
        
        if 'sentiment_std' in df.columns:
            df['sentiment_x_surprise'] = df['sentiment_std'] * df['surprise_std']
    
    # Sentiment disagreement
    if 'pre_ea_sentiment_std' in df.columns:
        df['disagreement_std'] = standardize(df['pre_ea_sentiment_std'])
        df['high_disagreement'] = (df['disagreement_std'] > 0.5).astype(int)
    
    # Triple interaction: Sentiment × EQ × Surprise
    if all(c in df.columns for c in ['sentiment_std', 'eq_std', 'surprise_std']):
        df['sentiment_x_eq_x_surprise'] = df['sentiment_std'] * df['eq_std'] * df['surprise_std']
    
    print(f"Created {len([c for c in df.columns if '_x_' in c or '_std' in c])} analysis variables")
    
    return df

# Create analysis variables
analysis_sample = create_analysis_variables(analysis_sample)

## 9. Save Final Dataset

In [ ]:
# =============================================================================
# SAVE FINAL DATASET
# =============================================================================

def save_final_dataset(df: pd.DataFrame, output_dir: str):
    """Save final analysis dataset with documentation."""
    
    os.makedirs(output_dir, exist_ok=True)
    
    # Main dataset (Parquet)
    filepath = os.path.join(output_dir, 'analysis_sample.parquet')
    df.to_parquet(filepath, index=False)
    print(f"Saved: {filepath}")
    
    # CSV for Stata/other software
    csv_path = os.path.join(output_dir, 'analysis_sample.csv')
    df.to_csv(csv_path, index=False)
    print(f"Saved: {csv_path}")
    
    # Data dictionary
    data_dict = {
        'Identifiers': {
            'ticker': 'Stock ticker symbol',
            'ea_date': 'Earnings announcement date'
        },
        'Pre-EA Social Media': {
            'pre_ea_posts': 'Total posts in pre-EA window [-10,-2]',
            'pre_ea_users': 'Unique users in pre-EA window',
            'pre_ea_attention': 'Log(1 + pre_ea_posts)',
            'pre_ea_sentiment_mean': 'Average FinBERT polarity',
            'pre_ea_pos_share': 'Share of positive posts',
            'pre_ea_neg_share': 'Share of negative posts',
            'pre_ea_disagreement': 'Average sentiment std (disagreement)',
            'sentiment_std': 'Standardized pre-EA sentiment'
        },
        'Returns (CARs)': {
            'CAR_m1_p1': 'Market-adjusted CAR[-1,+1] (EA window)',
            'CAR_0_p2': 'Market-adjusted CAR[0,+2]',
            'CAR_p2_p20': 'Market-adjusted CAR[+2,+20] (drift)',
            'CAR_m10_m2': 'Market-adjusted CAR[-10,-2] (pre-EA)',
            'ea_volatility': 'Return volatility around EA (annualized)'
        },
        'Earnings Quality': {
            'earnings_quality_dd_std': 'DD model quality (standardized)',
            'earnings_quality_mcn_std': 'McNichols model quality (standardized)',
            'earnings_quality_composite': 'Composite EQ measure',
            'eq_quintile': 'EQ quintile (Q1_Low to Q5_High)',
            'eq_high': 'High EQ indicator (Q4-Q5)',
            'eq_low': 'Low EQ indicator (Q1-Q2)',
            'eq_std': 'Standardized composite EQ'
        },
        'Earnings Surprise': {
            'eps_estimate': 'Consensus EPS estimate',
            'reported_eps': 'Reported EPS',
            'surprise_pct': 'Earnings surprise (%)',
            'surprise_std': 'Standardized surprise',
            'surprise_positive': 'Positive surprise indicator',
            'surprise_negative': 'Negative surprise indicator'
        },
        'Controls': {
            'price_pre_ea': 'Stock price before EA',
            'log_price': 'Log stock price',
            'volume_pre_ea': 'Trading volume before EA',
            'volatility_pre_ea': 'Pre-EA volatility (annualized)',
            'ret_1m/3m/6m': 'Prior cumulative returns',
            'sector': 'GICS sector',
            'ea_year/quarter/month': 'Time indicators'
        },
        'Interactions': {
            'sentiment_x_eq': 'Sentiment × EQ interaction',
            'attention_x_eq': 'Attention × EQ interaction',
            'sentiment_x_surprise': 'Sentiment × Surprise interaction',
            'sentiment_x_eq_x_surprise': 'Triple interaction'
        }
    }
    
    dict_path = os.path.join(output_dir, 'analysis_data_dictionary.json')
    with open(dict_path, 'w') as f:
        json.dump(data_dict, f, indent=2)
    print(f"Saved: {dict_path}")
    
    # Summary statistics
    summary = {
        'sample_size': len(df),
        'unique_firms': int(df['ticker'].nunique()),
        'date_range': [str(df['ea_date'].min()), str(df['ea_date'].max())],
        'variables': list(df.columns),
        'created_at': datetime.now().isoformat()
    }
    
    summary_path = os.path.join(output_dir, 'analysis_summary.json')
    with open(summary_path, 'w') as f:
        json.dump(summary, f, indent=2, default=str)
    print(f"Saved: {summary_path}")

# Save final dataset
save_final_dataset(analysis_sample, config.FINAL_DATA_PATH)

## 10. Summary

In [ ]:
# =============================================================================
# NOTEBOOK SUMMARY
# =============================================================================

print("""
╔══════════════════════════════════════════════════════════════════╗
║       NOTEBOOK 5: DATA MERGING COMPLETE                          ║
╚══════════════════════════════════════════════════════════════════╝

FINAL DATASET STRUCTURE:
────────────────────────
Unit of Observation: Firm-Earnings Announcement Event

Variable Blocks:
  1. Pre-EA WSB Sentiment and Attention
     • Posts, users, sentiment scores
     • Aggregated over [-10, -2] trading days

  2. Earnings Announcement CARs
     • CAR[-1,+1]: EA announcement return
     • CAR[+2,+20]: Post-EA drift

  3. Earnings Quality
     • Dechow-Dichev measure
     • McNichols modification
     • Composite score

  4. Earnings Surprise
     • Actual vs. expected EPS
     • Surprise direction

  5. Control Variables
     • Size, momentum, volatility
     • Sector, timing

  6. Interaction Terms
     • Sentiment × EQ
     • Attention × EQ
     • Triple interactions

OUTPUT FILES:
─────────────
• analysis_sample.parquet        - Main analysis dataset
• analysis_sample.csv            - CSV version
• analysis_data_dictionary.json  - Variable definitions
• analysis_summary.json          - Summary statistics

NEXT STEPS:
───────────
→ Notebook 6: Empirical Analysis & Regression Tests
  - Baseline pricing regressions
  - EQ interaction analysis
  - Reversal/mispricing tests
  - Robustness checks

""")

print(f"Final sample: {len(analysis_sample):,} observations")
print(f"Unique firms: {analysis_sample['ticker'].nunique()}")
print(f"Columns: {len(analysis_sample.columns)}")